<a href="https://colab.research.google.com/github/emyesme/CalcificationDetection/blob/Emily/Counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
 from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import pandas as pd
import os
#!pip install fastprogress
from fastprogress import master_bar, progress_bar
#!pip install tqdm
from tqdm.notebook import tqdm_notebook

In [18]:

def calculateFROC(groundTruthsDataFrame, normals, candidates):
  # evaluation froc curve #

  #counting
  evaluationList = []

  fn = 0 # false negative, for the blobs that do not belong to any component

  positive_candidates = 0
  flag = False

  for index, row in tqdm_notebook(groundTruthsDataFrame.iterrows()):

    imageKey = row['name'].split('_')[0]

    # list of features found with y,x and sigma
    candidatesImg = candidates.loc[candidates['name'].str.contains(imageKey, regex=False)]

    candidates_number = len(candidatesImg)
          
    for gtComponent in range(int(row.num_labels)):
      matches = []
      evaluationList = []
      
      for index2, candidate in candidatesImg.iterrows():

        if (imageKey in normals):
          continue
        # is the image a normal image?

        stat = eval(row.stats)[gtComponent]

        # top left is the 0,0 of the image
        
        left = stat['leftmost_x']
        right = stat['leftmost_x'] + stat['hor_size']

        top = stat['topmost_y']
        bottom = stat['topmost_y'] + stat['vert_size']

        if (left == 0) and (top ==0) and (gtComponent==0): #This is background so we can skip it
          continue

        if ( candidate.x >= left ) and ( candidate.x <= right ) and ( candidate.y >= top ) and ( candidate.y <= bottom ): #This is to find if the candidate is whithin the groundtruth component
          matches.append(candidate.x)

      if len(matches) == 1:
        positive_candidates += 1
      elif len(matches) > 1:
        positive_candidates += 1
      elif gtComponent > 0: #The component is not background
        fn = fn+1

    evaluationList.append([imageKey, int(row.num_labels)-1, candidates_number, positive_candidates])
    positive_candidates = 0

    dfEvaluationList= pd.DataFrame(evaluationList, columns=['key', 'num_labels_gt', 'candidates','positive candidates'])

    flag = writeFile(dfEvaluationList, flag)

    del dfEvaluationList
    del evaluationList


In [19]:
def writeFile(df, flag):
  if(flag):
    df.to_csv(os.path.join('/content',
                                 'drive',
                                 'MyDrive',
                                 'Results',
                                 'countpipeline8.csv'),
                    mode='a',
                    index=False)
    flag = False
  else:
    df.to_csv(os.path.join('/content',
                                 'drive',
                                 'MyDrive',
                                 'Results',
                                 'countpipeline8.csv'),
                  mode='a',
                  header=False,
                  index=False)
  return flag

In [ ]:
# from everything
RESULTS_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Results', 
                        '8.dehazingDil+DoG+GLCM')
                        #'4.dehazingDC+DoH(0.0005)+GLCM')

DATA_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Image Processing and Analysis 2022',
                        'projects',
                        'Calcification Detection',
                        'dataset')

gt = pd.read_csv(os.path.join('/content',
                              'drive',
                              'MyDrive',
                              'Results',
                              'groundTruthStats.csv'))

# read numbers of normal images
normals = []
with open(os.path.join(DATA_DIR,'normals.txt')) as f:
    for line in f:
        normals.append(line[:-1])

results_file = os.listdir(RESULTS_DIR)  
dataset = os.listdir(os.path.join(DATA_DIR,"images"))
candidates = pd.DataFrame()

for result in results_file:
    fileCandidate = pd.read_csv(os.path.join(RESULTS_DIR,result))
    candidates = candidates.append(fileCandidate, ignore_index=True)

calculateFROC(gt, normals, candidates)
# df.to_csv(os.path.join('/content',
#                                 'drive',
#                                 'MyDrive',
#                                 'Results',
#                                 'countpipeline4.csv'))

In [ ]:
# RESTARTING FROM A PREVIOUS EXECUTION

already = pd.read_csv(os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Results',
                        'countpipeline8.csv'))

already = already['key'].map(str)
already = already.tolist()



RESULTS_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',
                        'Results', 
                        '8.dehazingDil+DoG+GLCM')
                        #'4.dehazingDC+DoH(0.0005)+GLCM')

DATA_DIR = os.path.join('/content',
                        'drive',
                        'MyDrive',  
                        'Image Processing and Analysis 2022',
                        'projects',
                        'Calcification Detection',
                        'dataset')

gt = pd.read_csv(os.path.join('/content',
                              'drive',
                              'MyDrive',
                              'Results',
                              'groundTruthStats.csv'))

gt = gt['name'].map(str).tolist()
all = []
for gtrow in gt:
  all.append(gtrow.split("_")[0])

missing =list(set(all)-set(already))

# read numbers of normal images
normals = []
with open(os.path.join(DATA_DIR,'normals.txt')) as f:
    for line in f:
        normals.append(line[:-1])

results_file = os.listdir(RESULTS_DIR)  
dataset = os.listdir(os.path.join(DATA_DIR,"images"))
candidates = pd.DataFrame()

gtShort = pd.DataFrame()

for image in dataset:
  if image.split('_')[0] in missing:
    
    gtRow = gt.loc[gt['name'].str.contains(image, regex=False)]
    gtShort = gtShort.append(gtRow)
    fileCandidate = pd.read_csv(os.path.join(RESULTS_DIR,image[:-4]+'_features.csv'))
    candidates = candidates.append(fileCandidate, ignore_index=True)
    del gtRow

# gtShort = gtShort.sample(n=20)

print(len(gtShort))

calculateFROC(gtShort, normals, candidates)
